# Import libraries

In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # 3D plotting toolkit
import statsmodels.api as sm
import scipy.stats as stats
import importlib.util
import sys

2025-08-14 11:25:09.066808: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-14 11:25:09.103356: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 11:25:09.352881: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-14 11:25:09.464691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755163509.580193    2077 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755163509.60

# Load data

In [2]:
from scripts.parse_vars import parse_variables
path_vars = f"../../geno_simulation.txt"
R_directory = f"../../rstudio_geno_simulation"
R_file = f"create_geno.R"

variables = parse_variables(path_vars)
if 'G' not in globals():
    G = int(variables['G'])
if 'L' not in globals():
    L = int(variables['L'])
if 'c' not in globals():
    c = int(variables['c'])
if 'k' not in globals():
    k = int(variables['k'])
if 'M' not in globals():
    M = float(variables['M'])
if 'HWE' not in globals():
    HWE = int(variables['HWE'])

if 'tools' not in globals():
    tools = ['PCA', 'abyss_counted', 'abyss', 'no_corr']


if 'scenarios' not in globals():
    scenarios = ['snp_effect',
                 'linear_continuous',
                 'non_linear_continuous',
                 'discrete_global',
                 'discrete_localized',
                 'mix_linear_continuous',
                 'mix_non_linear_continuous',
                 'mix_discrete_global',
                 'mix_discrete_localized']

if 'very_rare_threshold_L' not in globals():
    very_rare_threshold_L = float(variables['very_rare_threshold_L'])
if 'very_rare_threshold_H' not in globals():
    very_rare_threshold_H = float(variables['very_rare_threshold_H'])
if 'rare_threshold_L' not in globals():
    rare_threshold_L = float(variables['rare_threshold_L'])
if 'rare_threshold_H' not in globals():
    rare_threshold_H = float(variables['rare_threshold_H'])
if 'common_threshold_L' not in globals():
    common_threshold_L = float(variables['common_threshold_L'])
if 'common_threshold_H' not in globals():
    common_threshold_H = float(variables['common_threshold_H'])
if 'F' not in globals():
    F = float(variables['F'])

path_geno = f"simulation_data/G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/genotype"
geno = pd.read_pickle((f"{path_geno}/complete_inbred.pkl"))

In [3]:
nr_pcs = 15

In [4]:
path_geno = f"simulation_data/G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/genotype"
geno = pd.read_pickle((f"{path_geno}/complete_inbred.pkl"))

In [5]:
geno_pcs = pd.read_pickle(f"simulation_data/G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/PCs/geno_{nr_pcs}_PCs.pkl")

In [7]:
# Create the three new DataFrames
minor = (geno == -1).astype(int)
het = (geno == 0).astype(int)
major = (geno == 1).astype(int)

In [14]:
def train_and_reconstruct(df1, df2, learning_rate, epochs):
    # Safety: ensure float32 for TF
    X = df1.astype("float32").values
    y = df2.astype("float32").values
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    n_out = y.shape[1]
    inputs = tf.keras.Input(shape=(X.shape[1],), dtype=tf.float32)
    outputs = tf.keras.layers.Dense(n_out, activation="linear", use_bias=True)(inputs)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss="mse",
    metrics=["mae"]
    )
    
    # (Optional) Early stopping to avoid overfitting/futile training
    cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )
    
    history = model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        epochs=epochs,           # will stop early if val_loss stops improving
        batch_size=64,
        callbacks=[cb],
        verbose=0
    )
    
    # Reconstruction: pass df1 through the trained model to get df3
    y_hat = model.predict(X, verbose=0)
    
    # Make it a DataFrame aligned to df2
    df3 = pd.DataFrame(y_hat, index=df2.index, columns=df2.columns)
    return df3

In [15]:
test = train_and_reconstruct(geno_pcs, geno, learning_rate=1e-3, epochs=500)

E0000 00:00:1755164082.488146    2077 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1755164082.490509    2077 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [16]:
test

,C_1_MAF_0.500,C_2_MAF_0.500,C_3_MAF_0.500,C_4_MAF_0.500,C_5_MAF_0.500,C_6_MAF_0.500,C_7_MAF_0.499,C_8_MAF_0.499,C_9_MAF_0.499,C_10_MAF_0.499,...,VR_3991_MAF_0.045,VR_3992_MAF_0.044,VR_3993_MAF_0.044,VR_3994_MAF_0.044,VR_3995_MAF_0.044,VR_3996_MAF_0.043,VR_3997_MAF_0.043,VR_3998_MAF_0.042,VR_3999_MAF_0.042,VR_4000_MAF_0.042
0,0.932436,0.985062,-0.980524,0.980904,0.164617,-0.806609,1.021576,0.542305,-0.915802,-0.146237,...,0.905629,0.951485,0.812777,0.946560,0.954336,0.960169,0.860231,0.864649,0.902791,0.849696
1,0.954313,0.992409,-1.138357,1.141313,0.103521,-0.827571,1.040792,0.528055,-0.933771,-0.177547,...,0.908128,0.956216,0.814037,0.946502,0.952263,0.966424,0.865878,0.861517,0.905229,0.849045
2,0.962530,0.992756,-1.041633,1.041104,0.127244,-0.827930,1.021251,0.539022,-0.914586,-0.122743,...,0.907006,0.957767,0.809152,0.949584,0.957746,0.965056,0.860851,0.865164,0.906298,0.853942
3,0.900433,0.985921,-1.123151,1.120053,0.117274,-0.827680,1.022500,0.557766,-0.916756,-0.159022,...,0.907104,0.955266,0.814082,0.949411,0.952244,0.962304,0.863582,0.865226,0.902385,0.850256
4,0.940234,0.976670,-1.103103,1.108710,0.093840,-0.830901,1.009315,0.480960,-0.896421,-0.104156,...,0.909443,0.956429,0.808605,0.946609,0.953118,0.965002,0.858821,0.870984,0.907110,0.850915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-1.254044,0.790539,0.607681,-0.670014,0.947621,-0.435102,-0.805045,0.741629,0.832415,-0.905892,...,1.005552,0.881925,0.857899,0.790211,0.783473,0.900254,0.982862,0.860505,0.880035,0.798635
1996,-1.283465,0.801439,0.690491,-0.756476,1.021555,-0.419390,-0.797141,0.824631,0.822034,-0.957844,...,1.007408,0.880569,0.857687,0.788546,0.780116,0.896531,0.983100,0.856436,0.876788,0.797795
1997,-1.251612,0.858213,0.565600,-0.629905,0.888276,-0.512219,-0.790045,0.715869,0.812495,-0.865896,...,1.000535,0.878441,0.863163,0.798876,0.789633,0.900649,0.980591,0.861390,0.877079,0.801330
1998,-1.261379,0.811329,0.647916,-0.706631,0.946646,-0.487074,-0.772371,0.734659,0.795792,-0.932292,...,1.006856,0.881504,0.858005,0.794158,0.786771,0.900178,0.980558,0.859059,0.877775,0.802959


In [17]:
nrs_pcs = list(set([int(f.split("_")[1]) for f in os.listdir(f"simulation_data//G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/PCs")]))

In [18]:
nrs_pcs

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [20]:
os.makedirs(f"simulation_data/G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/maf_reconstruct/",exist_ok = True)

In [ ]:
for nr_pcs in nrs_pcs:
    print(nr_pcs)
    geno_pcs = pd.read_pickle(f"simulation_data/G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/PCs/geno_{nr_pcs}_PCs.pkl")
    geno_reconstruct = train_and_reconstruct(geno_pcs, geno, learning_rate=1e-3, epochs=500)
    min_reconstruct = train_and_reconstruct(minor, geno, learning_rate=1e-3, epochs=500)
    het_reconstruct = train_and_reconstruct(het, geno, learning_rate=1e-3, epochs=500)
    major_reconstruct = train_and_reconstruct(major, geno, learning_rate=1e-3, epochs=500)
    geno_reconstruct.to_pickle(f"simulation_data/G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/maf_reconstruct/geno_reconstruct_{nr_pcs}_PCs.pkl")
    min_reconstruct.to_pickle(f"simulation_data/G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/maf_reconstruct/minor_reconstruct_{nr_pcs}_PCs.pkl")
    het_reconstruct.to_pickle(f"simulation_data/G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/maf_reconstruct/het_reconstruct_{nr_pcs}_PCs.pkl")
    major_reconstruct.to_pickle(f"simulation_data/G{G}_L{L}_c{c}_k{k}_M{M}_F{F}/maf_reconstruct/major_reconstruct_{nr_pcs}_PCs.pkl")

1
